In [1]:
import pandas as pd
import glob

pd.options.mode.chained_assignment = None
# read files
allfiles = glob.glob('Kickstarter_data/*.csv')
df = pd.concat((pd.read_csv(f) for f in allfiles), ignore_index=True)

Data is in multiple files, so simply read all and concatenate to single dataframe.

In [2]:
# Examine shape
print(df.shape)

# Examine keys
print(df.keys())

# Examine state counts
print(df.state.value_counts())

# Examine head to get an idea of data
df.head()

(195614, 37)
Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'fx_rate', 'goal', 'id', 'is_starrable',
       'launched_at', 'name', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type', 'location',
       'friends', 'is_backing', 'is_starred', 'permissions'],
      dtype='object')
successful    105680
failed         73634
canceled        8823
live            6876
suspended        601
Name: state, dtype: int64


,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,location,friends,is_backing,is_starred,permissions
0,80,I will be an artist-in-residence at Elsewhere ...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",3596,US,1325980505,"{""urls"":{""web"":{""user"":""https://www.kickstarte...",USD,$,True,...,1334866560,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",3596.0,domestic,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",NaN,NaN,NaN,NaN
1,47,We are looking to bring a Visiting Sculptor fr...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",4117,US,1330727362,"{""urls"":{""web"":{""user"":""https://www.kickstarte...",USD,$,True,...,1334959598,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",4117.0,domestic,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",NaN,NaN,NaN,NaN
2,80,Surrealistic oil paintings capturing the metam...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",3125,US,1332598567,"{""urls"":{""web"":{""user"":""https://www.kickstarte...",USD,$,True,...,1334635143,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",3125.0,domestic,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",NaN,NaN,NaN,NaN
3,82,1000 Artists is a public art-making installati...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",4586,US,1330926084,"{""urls"":{""web"":{""user"":""https://www.kickstarte...",USD,$,True,...,1336440145,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",4586.0,domestic,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",NaN,NaN,NaN,NaN
4,31,P.M.A.F.T.W. my upcoming solo show June 2012 a...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",1036,US,1332476607,"{""urls"":{""web"":{""user"":""https://www.kickstarte...",USD,$,True,...,1333421843,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",1036.0,domestic,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",NaN,NaN,NaN,NaN


In [3]:
# Check for duplicate projects

print(len(set(df.id)), len(df.id))

177768 195614


There seems to be duplicate projects. Solution for this is drop all but first occurence

In [4]:
df.drop_duplicates(subset=['id'], inplace=True)
print(df.shape)

(177768, 37)


Next task is to preprocess values and drop irrelevant features.

First convert all goals to be USD.

In [5]:
df['goal'] = df.apply(lambda row: row['goal'] * row['static_usd_rate'], axis=1)

Filter out unnecessary keys that contain either irrelevant or duplicate information
* Friends, is_backing, is_starred and permissions are irrelevant, because they do not hold any information about project
* Currency symbol, trailing code, current currency, static_usd_rate and usd_type can also be dropped. Currency is enough.
* Pledged and converted_pledged_amount can be dropped, because we are interested in USD_pledged
* Spotlight is only for successful projects. We found out this after our decision tree had only 1 node: Spotlight: true or false
* Slug, photo, profile have only urls, so drop them
* id is numeric id for projects. Not relevant
* Created_at and state_changed_at are dropped, because we are interested in launched_at and deadline
* URLS are not dropped, because they can be used to check state from live projects


In [6]:
# Filter out unnecessary keys that contain either irrelevant or duplicate information
# Such as url for photo, pledged amount in non-USD -currency, etc.
# URLS are not dropped, because they can be used to check state from live projects
# Also spotlight should be dropped because it is only available for finished projects
unnecessary_keys = ['currency_symbol', 'currency_trailing_code', 'current_currency', 'id', 'name', 'fx_rate',
                    'photo', 'pledged', 'profile', 'slug', 'source_url', 'static_usd_rate', 'state_changed_at', 'created_at',
                    'usd_type', 'friends', 'is_backing', 'is_starred', 'permissions', 'converted_pledged_amount', 'spotlight']

df.drop(unnecessary_keys, axis=1, inplace = True)
print(df.keys())

Index(['backers_count', 'blurb', 'category', 'country', 'creator', 'currency',
       'deadline', 'disable_communication', 'goal', 'is_starrable',
       'launched_at', 'staff_pick', 'state', 'urls', 'usd_pledged',
       'location'],
      dtype='object')


Some of the features are stored as JSON string. Filter them so they are easier to read.

* Convert category to example Art/Taxidermy. Another solution would be to split it to two: top category: Art and category: Taxidermy
* Convert creator to numeric ID
* Convert location to be type Anaheim, CA
* Filter only relevant urls (project and rewards)

In [7]:
import json

def filter_json(val, key):
    cat = json.loads(val)
    return cat[key]

# Category, name, location
df['category'] = df.apply(lambda row: filter_json(row['category'], 'slug'), axis = 1)
df['creator'] = df.apply(lambda row: filter_json(row['creator'], 'id'), axis = 1)
df['location'] = df.apply(lambda row: filter_json(row['location'], 'displayable_name') if type(row['location']) != float else 'unknown', axis = 1)
df['urls'] = df.apply(lambda row: filter_json(row['urls'], 'web'), axis = 1)

Blurb is short description of the project. While it would be interesting to analyze it, we are now only interested in length.

In [8]:
df['blurb_length'] = df.apply(lambda row: len(row['blurb'].split()) if type(row['blurb']) != float else 0, axis=1)

Count the duration of project to be the number of days between launched_at and deadline, round up.

In [9]:
import datetime

def count_days(start, end):
    duration = datetime.datetime.fromtimestamp(end) - datetime.datetime.fromtimestamp(start)
    return duration.days

df['duration'] = df.apply(lambda row: count_days(row.launched_at, row.deadline), axis=1)

Drop replaced keys and separate finished and unfinished projects

In [10]:
df.drop(['blurb', 'launched_at', 'deadline'], axis=1, inplace=True)
finished = df.loc[df.state != 'live']
unfinished = df.loc[df.state == 'live']
print(finished.shape)
print(unfinished.shape)

(173823, 15)
(3945, 15)


Because projects can be canceled or suspended for varioous reasons, drop them from our data.
Also convert successful projects to 1 and failed to 0

In [11]:
finished = finished[finished.state != 'canceled']
print(finished.state.value_counts())
finished = finished[finished.state != 'suspended']
print(finished.state.value_counts())
finished['state'][finished.state != 'successful'] = 0
finished['state'][finished.state == 'successful'] = 1
print(finished.state.value_counts())

successful    91355
failed        73044
suspended       601
Name: state, dtype: int64
successful    91355
failed        73044
Name: state, dtype: int64
1    91355
0    73044
Name: state, dtype: int64


In [12]:
# save data as csv-file
finished.to_csv('finished.csv', index = False)
unfinished.to_csv('unfinished.csv', index = False)